In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/squad-test/dev-v1.1.json
/kaggle/input/squad-dataset/train-v1.1.json
/kaggle/input/mental-health-faq-for-chatbot/Mental_Health_FAQ.csv


In [3]:
# Load and preprocess the dataset
import pandas as pd
df = pd.read_csv('/kaggle/input/mental-health-faq-for-chatbot/Mental_Health_FAQ.csv')

df

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."
...,...,...,...
93,4373204,How do I know if I'm drinking too much?,Sorting out if you are drinking too much can b...
94,7807643,"If cannabis is dangerous, why are we legalizin...","Cannabis smoke, for example, contains cancer-c..."
95,4352464,How can I convince my kids not to use drugs?,You can't. But you can influence their capacit...
96,6521784,What is the legal status (and evidence) of CBD...,Cannabidiol or CBD is a naturally occurring co...


In [4]:
train_data = []

for i in range(len(df)):
    row = df.iloc[i]
    context = row['Answers']
    if i < 10:
        id_no = "0000" + str(i + 1)
    else:
        id_no = "000" + str(i + 1)

    temp = {}
    temp['context'] = row['Answers']
    temp['qas'] = [{
        'id' : str(row['Question_ID']),
        'is_impossible' : False,
        'question' : row['Questions'],
        'answers' : [
            {
                'text' : row['Answers'][:context.index(".")+1],
                'answer_start' : 0
            }
        ]
    }]
    
    train_data.append(temp)

In [5]:
train_data[0]

{'context': 'Mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.\nMental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital.\nIt is important to know that mental illnesses are medical conditions that have nothing to do with a personâ€™s character, intelligence, or willpower. Just as diabetes is a disorder of the pancreas, mental illness is a medical condition due to the brainâ€™s biology.\nSimilarly to how one would treat diabetes with medi

In [6]:
!pip install simpletransformers

  Obtaining dependency information for simpletransformers from https://files.pythonhosted.org/packages/26/9d/1030f946de5a9c312586b05c881194153fa34b54ca869dd9cba6a2bbad97/simpletransformers-0.64.5-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/e9/07/63a6e890c9b998a6318b46c2a34377fd1a3e01a94c427d82bfb2472b7c16/streamlit-1.30.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for validators<1,>=0.2 from https://files.pythonhosted.org/packages/3a/0c/785d317eea99c3739821718f118c70537639aa43f96bfa1d83a71f68eaf6/validators-0.22.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 43.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.5 MB/

In [7]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [8]:
model_type="distilbert"
model_name= "distilbert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

In [9]:
# Configure the model 
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size=3
model_args.num_train_epochs=5### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    "use_cuda":False,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [10]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args, use_cuda=False
)

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [11]:
# Train the model
model.train_model(train_data, eval_data=train_data[:25])

add example index and unique id: 100%|██████████| 98/98 [00:00<00:00, 358050.34it/s]


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

Running Epoch 3 of 5:   0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(10,
 {'global_step': [2, 4, 6, 8, 10],
  'correct': [1, 6, 6, 10, 10],
  'similar': [20, 15, 18, 15, 15],
  'incorrect': [4, 4, 1, 0, 0],
  'train_loss': [4.730408191680908,
   4.223745822906494,
   3.940640449523926,
   3.895317554473877,
   3.281553030014038],
  'eval_loss': [0.02834109589457512,
   -0.12581925094127655,
   -0.24937911331653595,
   -0.34224849939346313,
   -0.38868439197540283]})

In [12]:
# Evaluate the model
result, texts = model.eval_model(train_data[:50])

add example index and unique id: 100%|██████████| 50/50 [00:00<00:00, 56405.38it/s]


Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
result

{'correct': 20,
 'similar': 24,
 'incorrect': 6,
 'eval_loss': -0.3548852652311325}

## Making predictions

In [14]:
test_qs = []

for i in range(len(df)):
    row = df.iloc[i]
    context = row['Answers']
    question = row['Questions']
    
    temp = [{
        "context" : context,
        "qas" : [
            {
                "question" : question,
                "id" : str(i)
            }
        ]
    }]
    
    test_qs.append(temp)

In [25]:
import random

random.seed(47)

qs = random.choices(test_qs, k=10)

In [56]:
def answer_question(quest):
    answers, probabilities = model.predict(quest)
    
    print("Q: " + quest[0]["qas"][0]["question"])
    
    print("A: ", end="")
    for x in answers[0]["answer"]:
        if x != "":
            print(x, end="")
    print()

In [29]:
for question_ in qs:
    answer_question(question_)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 916.19it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Q: How can I tell if health information is good or bad?
A: Anyone can post anything online.Anyone can post anything online. It's up to the readers to think critically about information to decide if it seems helpful, truthful, or safe. Here are key points to keep in mind as you determine which information is worth considering and which information doesn't pass the test.hand, factual statements like "We have 50 new cases in BC" give you important information without playing into your emotions.


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2115.13it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Q: What causes mental illness?
A: It is estimated that mental illness affects 1 in 5 adults in America, and that 1 in 24 adults have a serious mental illness.It is estimated that mental illness affects 1 in 5 adults in America, and that 1 in 24 adults have a serious mental illness. Mental illness does not discriminate; it can affect anyone, regardless of gender, age, income, social status, ethnicity, religion, sexual orientation, or background. Although mental illness can affect anyone, certain conditions may be more common in different populations.


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1343.04it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Q: How can I find a mental health professional right for my child or myself?
A: Feeling comfortable with the professional you or your child is working with is critical to the success of your treatment. Finding the professional who best fits your needs may require some research.Feeling comfortable with the professional you or your child is working with is critical to the success of your treatment.


## Creating new questions

In [31]:
random_contexts = random.choices(df["Answers"], k=5)

for r in random_contexts:
    print(r)
    print("----------------------------")

It can be difficult to find the things that will help you, as different things help different people. It's important to be open to a range of approaches and to be committed to finding the right help and to continue to be hopeful, even when some things don't work out.
----------------------------
Cannabis smoke, for example, contains cancer-causing toxins. However, the risk of developing some cancers (e.g., mouth, tongue and lung) is less for cannabis smokers than tobacco smokers, partly because they tend to smoke less than tobacco users. And, while all drugs have an effect on the brain, the particular properties of the drug influence the level of risk of harmful consequences. The negative effects of cannabis on the brain, for example, seem to be less than the effects of some substances such as alcohol. 
 Legalizing cannabis provides an opportunity to put in place regulations to minimize potential harms. The danger of buying and using any illegal drug is that we can never know for sure 

In [32]:
new_questions = [
    "How do I find the right help for mental health issues?",
    "What are the benefits of legalizing cannabis?",
    "What are some ways to seek help for a child or youth's mental health?",
    "What is MSP?",
    "What are some early signs of psychosis?"
]

new_qs_dict = [
    [{
        "context" : random_contexts[0],
        "qas" : [{
            "question" : new_questions[0],
            "id" : "0"
        }]
    }],
    [{
        "context" : random_contexts[1],
        "qas" : [{
            "question" : new_questions[1],
            "id" : "1"
        }]
    }],
    [{
        "context" : random_contexts[2],
        "qas" : [{
            "question" : new_questions[2],
            "id" : "2"
        }]
    }],
    [{
        "context" : random_contexts[3],
        "qas" : [{
            "question" : new_questions[3],
            "id" : "3"
        }]
    }],
    [{
        "context" : random_contexts[4],
        "qas" : [{
            "question" : new_questions[4],
            "id" : "4"
        }]
    }]
]

In [33]:
for q in new_qs_dict:
    answer_question(q)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1849.34it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Q: How do I find the right help for mental health issues?
A: It can be difficult to find the things that will help you, as different things help different people. It's important to be open to a range of approaches and to be committed to finding the right help and to continue to be hopeful, even when some things don't work out.It can be difficult to find the things that will help you, as different things help different people.


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1316.89it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Q: What is MSP?
A: Mental health services like counselling or therapy may be covered by your BC Medical Services Plan (MSP) when you access the service through a hospital or mental health clinic.Mental health services like counselling or therapy may be covered by your BC Medical Services Plan (MSP) when you access the service through a hospital or mental health clinic. Find your local health authority to learn more about counselling services in your area. HealthLink BC’s Find Services tool and Locator app can also help you find services. You often need a doctor’s referral to access these types of programs.HealthLink BC’s Find Services tool and Locator app can also help you find services. You often need a doctor’s referral to access these types of programs. Without a doctor’s referral, you need to access care through the private system, which is not funded by the government.


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2192.53it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Q: What are some early signs of psychosis?
A: Prodrome is a medical term for early signs or symptoms of an illness or health problem that appear before the major signs or symptoms start. Psychosis, a group of symptoms found in disorders like schizophrenia, is one illness with a specific prodrome. With psychosis, prodrome signs and symptoms start before full symptoms like hallucinations and delusions appear.Prodrome is a medical term for early signs or symptoms of an illness or health problem that appear before the major signs or symptoms start. Psychosis, a group of symptoms found in disorders like schizophrenia, is one illness with a specific prodrome.Prodrome is a medical term for early signs or symptoms of an illness or health problem that appear before the major signs or symptoms start.


## DistilRoberta with altered training arguments

Changed the number of epochs from 5 to 20 (more epochs are thought to result in better predictions).

In [34]:
model_type = "roberta"
model_name = "distilroberta-base"

train_args["num_train_epochs"] = 20
train_args["output_dir"] = f"outputs/{model_type}"
train_args["best_model_dir"] = f"outputs/{model_type}/best_model"
train_args["wandb_kwargs"] = {"name": model_name}

In [35]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args, use_cuda=False
)

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [36]:
# Train the model
model.train_model(train_data, eval_data=train_data[:25])

add example index and unique id: 100%|██████████| 98/98 [00:00<00:00, 29699.55it/s]


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

correct,▁▅▅██
eval_loss,█▅▃▂▁
global_step,▁▃▅▆█
incorrect,██▃▁▁
similar,█▁▅▁▁
train_loss,█▆▄▄▁
correct,10
eval_loss,-0.38868
global_step,10
incorrect,0
similar,15


Running Epoch 0 of 20:   0%|          | 0/2 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 25/25 [00:00<00:00, 175.62it/s]A

add example index and unique id: 100%|██████████| 25/25 [00:00<00:00, 86373.64it/s]


...

Running Epoch 18 of 20:   0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 19 of 20:   0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(40,
 {'global_step': [2,
   4,
   6,
   8,
   10,
   12,
   14,
   16,
   18,
   20,
   22,
   24,
   26,
   28,
   30,
   32,
   34,
   36,
   38,
   40],
  'correct': [0,
   0,
   8,
   9,
   10,
   7,
   11,
   14,
   13,
   13,
   13,
   15,
   15,
   16,
   15,
   16,
   16,
   16,
   16,
   16],
  'similar': [2,
   7,
   17,
   16,
   15,
   18,
   14,
   11,
   12,
   12,
   12,
   10,
   10,
   9,
   10,
   9,
   9,
   9,
   9,
   9],
  'incorrect': [23, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'train_loss': [4.927789688110352,
   4.773817539215088,
   4.598581314086914,
   4.314167022705078,
   3.839465379714966,
   3.379836320877075,
   2.6900243759155273,
   1.9837093353271484,
   1.8040733337402344,
   1.5565550327301025,
   1.287855625152588,
   1.2888233661651611,
   1.1600985527038574,
   0.9376634359359741,
   1.206966519355774,
   1.2961270809173584,
   1.097057580947876,
   1.0990910530090332,
   1.1411893367767334,
   1.1841628551483154],
  'eval

In [37]:
for question_ in qs:
    answer_question(question_)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1555.75it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Q: How can I tell if health information is good or bad?
A: Anyone can post anything online. It's up to the readers to think critically about information to decide if it seems helpful, truthful, or safe. Here are key points to keep in mind as you determine which information is worth considering and which information doesn't pass the test. It's so easy to spread information online, and we can't assume that the source we're reading or watching is the original source.Anyone can post anything online.hand, factual statements like "We have 50 new cases in BC" give you important information without playing into your emotions.


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1985.00it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Q: What causes mental illness?
A: It is estimated that mental illness affects 1 in 5 adults in America, and that 1 in 24 adults have a serious mental illness.It is estimated that mental illness affects 1 in 5 adults in America, and that 1 in 24 adults have a serious mental illness. Mental illness does not discriminate; it can affect anyone, regardless of gender, age, income, social status, ethnicity, religion, sexual orientation, or background.


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1891.88it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Q: How can I find a mental health professional right for my child or myself?
A: Feeling comfortable with the professional you or your child is working with is critical to the success of your treatment.Feeling comfortable with the professional you or your child is working with is critical to the success of your treatment. Finding the professional who best fits your needs may require some research.


In [38]:
for q in new_qs_dict:
    answer_question(q)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2626.36it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Q: How do I find the right help for mental health issues?
A: It can be difficult to find the things that will help you, as different things help different people.It can be difficult to find the things that will help you, as different things help different people. It's important to be open to a range of approaches and to be committed to finding the right help and to continue to be hopeful, even when some things don't work out.


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2186.81it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Q: What is MSP?
A: Mental health services like counselling or therapy may be covered by your BC Medical Services Plan (MSP) when you access the service through a hospital or mental health clinic.Mental health services like counselling or therapy may be covered by your BC Medical Services Plan (MSP) when you access the service through a hospital or mental health clinic. Find your local health authority to learn more about counselling services in your area. HealthLink BC’s Find Services tool and Locator app can also help you find services.


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2257.43it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Q: What are some early signs of psychosis?
A: Prodrome is a medical term for early signs or symptoms of an illness or health problem that appear before the major signs or symptoms start.Prodrome is a medical term for early signs or symptoms of an illness or health problem that appear before the major signs or symptoms start. Psychosis, a group of symptoms found in disorders like schizophrenia, is one illness with a specific prodrome. With psychosis, prodrome signs and symptoms start before full symptoms like hallucinations and delusions appear.


## Combining our dataset with SQuAD dataset

In [39]:
import json

# Load the SQuAD dataset files
file_paths = ['/kaggle/input/squad-dataset/train-v1.1.json']

# Load and inspect the structure of the training dataset
with open(file_paths[0]) as file:
    squad_train = json.load(file)

with open('/kaggle/input/squad-test/dev-v1.1.json') as f:
    test = json.load(f)

In [40]:
def format_data(data):
    formatted_data = []
    for section in data['data']:
        for para in section['paragraphs']:
            context = para['context']
            for qa in para['qas']:
                temp = {}
                temp['context'] = context
                temp['qas'] = [qa]
                formatted_data.append(temp)
    return formatted_data

squad_train = format_data(squad_train)
test = format_data(test)

In [41]:
squad_train[0]

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'qas': [{'answers': [{'answer_start': 515,
     'text': 'Saint Bernadette Soubirous'}],
   'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
   'id': '5733be284776f41900661182'}]}

In [42]:
train_data[0]

{'context': 'Mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.\nMental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital.\nIt is important to know that mental illnesses are medical conditions that have nothing to do with a personâ€™s character, intelligence, or willpower. Just as diabetes is a disorder of the pancreas, mental illness is a medical condition due to the brainâ€™s biology.\nSimilarly to how one would treat diabetes with medi

In [43]:
len(squad_train)

87599

In [44]:
squad_train = random.choices(squad_train, k=1000)

In [45]:
len(squad_train)

1000

In [46]:
squad_train[0]

{'context': 'Between 1963 and 1965, there was a "downtown boom" that led to the construction of more than 700 buildings in the city. In 1968, Virginia Commonwealth University was created by the merger of the Medical College of Virginia with the Richmond Professional Institute. In 1970, Richmond\'s borders expanded by an additional 27 square miles (70 km2) on the south. After several years of court cases in which Chesterfield County fought annexation, more than 47,000 people who once were Chesterfield County residents found themselves within the city\'s perimeters on January 1, 1970. In 1996, still-sore tensions arose amid controversy involved in placing a statue of African American Richmond native and tennis star Arthur Ashe to the famed series of statues of Confederate heroes of the Civil War on Monument Avenue. After several months of controversy, the bronze statue of Ashe was finally completed on Monument Avenue facing the opposite direction from the Confederate Heroes on July 10, 1

In [47]:
len(test)

10570

In [48]:
test = random.choices(test, k=200)

In [49]:
for x in train_data:
    squad_train.append(x)
    test.append(x)

In [50]:
model_type = "roberta"
model_name = "distilroberta-base"

train_args["num_train_epochs"] = 3
train_args["output_dir"] = f"outputs/{model_type}"
train_args["best_model_dir"] = f"outputs/{model_type}/best_model"
train_args["wandb_kwargs"] = {"name": model_name}

In [51]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args, use_cuda=False
)

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [52]:
# Train the model
model.train_model(squad_train, eval_data=test)

add example index and unique id: 100%|██████████| 1098/1098 [00:00<00:00, 441040.59it/s]


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

correct,▁▁▅▅▅▄▆▇▇▇▇█████████
eval_loss,███▇▇▆▅▅▄▄▃▂▂▂▁▁▁▁▁▁
global_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
incorrect,█▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
similar,▁▃█▇▇█▆▅▅▅▅▅▅▄▅▄▄▄▄▄
train_loss,██▇▇▆▅▄▃▃▂▂▂▁▁▁▂▁▁▁▁
correct,16
eval_loss,-3.89406
global_step,40
incorrect,0
similar,9


Running Epoch 0 of 3:   0%|          | 0/9 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 298/298 [00:02<00:00, 131.74it/s]

add example index and unique id: 100%|██████████| 298/298 [00:00<00:00, 200048.43it/s]


...

Running Epoch 1 of 3:   0%|          | 0/9 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/9 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

(27,
 {'global_step': [9, 18, 27],
  'correct': [10, 1, 5],
  'similar': [132, 267, 211],
  'incorrect': [154, 28, 80],
  'train_loss': [4.631922721862793, 3.9840331077575684, 3.731954574584961],
  'eval_loss': [-0.22121575474739075,
   -1.2558484474817913,
   -1.4127471049626668]})

In [53]:
# Evaluate the model
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

In [54]:
result

{'correct': 5,
 'similar': 211,
 'incorrect': 80,
 'eval_loss': -1.4127471049626668}

In [57]:
for question_ in qs:
    answer_question(question_)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1096.55it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Q: How can I tell if health information is good or bad?
A: .emotions. Helpful news or information sources want you feel informed and empowered. Who is paying for the site? What products or services are being advertised? What links or other sites do they promote? Are you learning something from a story or experience, or are you simply being directed to buy a product or service? Social media can be a great way to keep in touch with family and friends, especially as we practice physical distancing or self-isolation.emotions. Helpful news or information sources want you feel informed and empowered.


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1692.62it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Q: What causes mental illness?
A: .


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1833.98it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Q: How can I find a mental health professional right for my child or myself?
A: .


In [58]:
for q in new_qs_dict:
    answer_question(q)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1608.25it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Q: How do I find the right help for mental health issues?
A: .


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1071.89it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Q: What is MSP?
A: Mental health services like counselling or therapy may be covered by your BC Medical Services Plan (MSP) when you access the service through a hospital or mental health clinic. Find your local health authority to learn more about counselling services in your area. HealthLink BC’s Find Services tool and Locator app can also help you find services.Mental health services like counselling or therapy may be covered by your BC Medical Services Plan (MSP) when you access the service through a hospital or mental health clinic.


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 767.48it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Q: What are some early signs of psychosis?
A: Prodrome is a medical term for early signs or symptoms of an illness or health problem that appear before the major signs or symptoms start. Psychosis, a group of symptoms found in disorders like schizophrenia, is one illness with a specific prodrome.
